In [7]:
import importlib

import RktnChallenge.RktnModel
importlib.reload(RktnChallenge.RktnModel)
from RktnChallenge.RktnModel import ModelTrainer

import RktnChallenge.preprocessing.translater
importlib.reload(RktnChallenge.preprocessing.translater)
from RktnChallenge.preprocessing.translater import LanguageDetector
from RktnChallenge.preprocessing.translater import Translator

import RktnChallenge.preprocessing.CopyWhereNan
importlib.reload(RktnChallenge.preprocessing.CopyWhereNan)
from RktnChallenge.preprocessing.CopyWhereNan import CopyWhereNan

import RktnChallenge.preprocessing.removeHTML
importlib.reload(RktnChallenge.preprocessing.removeHTML)
from RktnChallenge.preprocessing.removeHTML import removeHTML

On charge la base et init le support GPU / CPU

In [8]:
RktnModel = ModelTrainer("data.csv")

# On initialise le GPU disponible, la gestion mémoire
RktnModel.initGPU()

1 Physical GPUs, 1 Logical GPUs


On détecte la langue de la source et range dans une nouvelle colonne

In [9]:
desc_removeHTML = removeHTML("description")
desi_removeHTML = removeHTML("designation")

desi_langueDetect = LanguageDetector("desi_langue","designation",confiance=0.8)
desc_langueDetect = LanguageDetector("desc_langue","description",confiance=0.8)


desi_Translator = Translator(dest = "tr_designation", src="designation",               
                             detected_lang="desi_langue", source = "google",verbose = False)

desc_Translator = Translator(dest = "tr_description", src="description",               
                             detected_lang="desc_langue", source = "google", verbose = False)



RktnModel.initPreprocess()
# unidecode et removeHTML
RktnModel.addPreprocessStep("desc removeHTML",desc_removeHTML)
RktnModel.addPreprocessStep("desi removeHTML",desi_removeHTML)
# decode la langue et traduit la designation
RktnModel.addPreprocessStep("detect langue designation",desi_langueDetect)
RktnModel.addPreprocessStep("translate designation",desi_Translator)
# decode la langue et traduit la description
RktnModel.addPreprocessStep("detect langue description",desc_langueDetect)
RktnModel.addPreprocessStep("translate description",desc_Translator)
RktnModel.preprocess()


print("langues detectees dans les designations: ",desi_langueDetect.langues)
print("langues detectees dans les descriptions: ",desc_langueDetect.langues)

/home/pec/Rktn/RktnChallenge/preprocessing/removeHTML.py:17: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  X[self.dest] = X[self.dest].apply(lambda x: "" if(pd.isna(x)) else unidecode(BeautifulSoup(x, "lxml").text))
/home/pec/Rktn/RktnChallenge/preprocessing/removeHTML.py:17: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  X[self.dest] = X[self.dest].apply(lambda x: "" if(pd.isna(x)) else unidecode(BeautifulSoup(x, "lxml").text))
/home/pec/Rktn/RktnChallenge/preprocessing/removeHTML.py:17: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  X[self.dest] = X[self.dest].apply(lambda x: "" if(pd.isna(x)) else unidecode

Loading lid Lang model


fin de detection
translating ... designation
erreur:  Hallwag Carte Routiere Deutschland 
erreur:  Totika Set Five Card Decks Included
erreur:  S.I.C. Kamen Rider Zx (Completed) Bandai [Japan]  7""
erreur:  Jahrbuch Der Luftwaffe / Folge 12
erreur:  Guitar Hero 2 Bundle With Guitar
erreur:  Coprirete In Feltro Cod.020-7100
erreur:  Dr. Whostealth Cyberman 6th. Doctor
erreur:  VijverTechniek (VT) Filtre de bassin Vex-200
erreur:  Leitz 50 X Pendelhefter Orgacolor A4 Kaufmannische Heftung
fin de translation
fin de detection
translating ... description
erreur:  p7 inch plush features Hello Kitty dressed as an Ugly Doll.p
erreur:  pEquipment and furniture for playing in the kitchen of Amandas house. Made of farmed wood. Includes stove with oven sink with cupboard table with two stools coffee machine and toaster. Furnish the entire house and collect all the members of this playful family. Wood from repopulated forests. For each tree felled three more are planted. Amanda and her troupe are a

On vient de traduire dans tr_ ce qui n'est pas en francais, on complete avec le français:

In [12]:
# on copie les lignes qui n'ont pas été traduites (celles en français)
desc_copy = CopyWhereNan("tr_description","description")
desi_copy = CopyWhereNan("tr_designation","designation")

RktnModel.initPreprocess()
RktnModel.addPreprocessStep("desc copy french",desc_copy)
RktnModel.addPreprocessStep("desi copy french",desi_copy)
RktnModel.data = RktnModel.preprocess()


#on sauve
RktnModel.saveCSV("data_tr.csv")
